# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply



import app



if __name__ == "__main__":

    
    app.application.run()



 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Sep/2017 01:47:35] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2017 01:47:44] "POST /main HTTP/1.1" 200 -


In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

In [ ]:
import datetime
now = datetime.datetime.now()
str(now.year)+str(now.month if now.month >= 10 else  '0'+str(now.month))+str(now.day if now.day >= 10 else  '0'+str(now.hour)) +' '+str(now.hour if now.hour >= 10 else  '0'+str(now.hour)) + str(now.minute if now.minute >= 10 else  '0'+str(now.minute)) + str(now.second if now.second >= 10 else  '0'+str(now.second))

In [ ]:
from methods import access
from methods import psql_methods
import requests
import json

import psycopg2
from pandas import DataFrame
chat_id =84723474
reply_markup_main = {'keyboard': [['/expense_add'],['/crypto ETH-USD']], 'resize_keyboard': True, 'one_time_keyboard': False}

url = 'https://api.cryptonator.com/api/ticker/eth-usd'

r = requests.get(url)

dict_crypto_curse = json.loads(r.text)

ETH_USD = round(float(dict_crypto_curse['ticker']['price']), 3)
text = str(ETH_USD) + ' $ за дозу эфирчика...'
reply_markup = reply_markup_main


In [2]:
#тест отправки графика
import psycopg2
from pandas import DataFrame
import pandas as pd
from datetime import datetime
from datetime import timedelta
dbname = 'dbr3jigs1op5oo'
port = '5432'
user = 'muwrkppfuyldmk'
host = 'ec2-54-227-252-202.compute-1.amazonaws.com'
password = '4c4eabfcaf92f7289ccfc1a314d04a3c3806db72b1bf12fd5f0f40c410b14355'

PSQL_heroku_keys = {'dbname' : dbname
                    , 'port' : port
                    , 'user' : user
                    , 'host' : host
                    , 'password' : password
                    }



user_id = 13


conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)

cur = conn.cursor()
#достаем информацию по пользователю
cur.execute("SELECT * from public.transaction_fact where user_id = %(user_id)s" % {'user_id' : user_id} )
df = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
cur.close()

In [3]:

if df.shape[0] != 0: 
    #получаем необходимые даты
    cur_year = datetime.now().year
    cur_month = datetime.now().month

    prev_year = cur_year if cur_month > 1 else cur_year -1
    prev_month = cur_month - 1 if cur_month > 1 else 12

    next_year = cur_year if cur_month < 12 else cur_year + 1
    next_month = cur_month + 1 if cur_month < 12 else 1

    #получаем df за текущий и предыдущий месяцы
    df_cur_month_expense = df[(df.date_fact >= pd.datetime(cur_year,cur_month,1)) & (df.date_fact < pd.datetime(next_year,next_month,1))][:] 
    df_all_prev_month_expense = df[(df.date_fact < pd.datetime(cur_year,cur_month,1))][:] 

    #средние траты за все время по месяцам
    df_all_prev_month_expense_group = DataFrame(data = df_all_prev_month_expense.groupby(['transaction_type'])['summa'].sum() ).sort_values(by='summa' , ascending=False)

    df_all_prev_month_expense['year'] = df_all_prev_month_expense['date_fact'].dt.year
    df_all_prev_month_expense['month'] = df_all_prev_month_expense['date_fact'].dt.month
    df_all_prev_month_expense['yearmonth'] = df_all_prev_month_expense['year'].astype(str) +  df_all_prev_month_expense['month'].astype(str)

    df_all_prev_month_expense_group = DataFrame(df_all_prev_month_expense.groupby(['transaction_type' , 'yearmonth'])['summa'].sum() ).reset_index()
    df_all_prev_month_expense_group['summa'] = df_all_prev_month_expense_group['summa'].astype(float)
    df_mean_expense = DataFrame(df_all_prev_month_expense_group.groupby(['transaction_type'])['summa'].mean()).reset_index()

    
    #отсортированные и сгрупированные траты в текущем и предыдущем месяце
    df_cur_month_expense_group = DataFrame(data = df_cur_month_expense.groupby(['transaction_type'])['summa'].sum() ).sort_values(by='summa' , ascending=False)


    #заводм справочник популярных трат
    type_expand = ([])
    for row in df_cur_month_expense_group.iterrows():
        k  = 1
        type_expand.append(row[0])

    #суммы трат
    sum_expand = df_cur_month_expense_group.values.astype(float)
    all_sum = df_cur_month_expense_group['summa'].values.astype(float).sum()

    text = 'Ваши траты в текущем месяце: \n' 
    for type , sum in zip(type_expand , sum_expand):
        if df_mean_expense[(df_mean_expense.transaction_type == type)].shape[0] == 0:
            mean_change = ''
        else:
            change = sum / df_mean_expense[(df_mean_expense.transaction_type == type)]['summa'].values[0]

            sign = 'less' if change < 1. else 'more'
            str_change_vlue = str(int( (1. - change if sign == 'less' else change - 1.) * 100 ))
            mean_change = '( на ' + str_change_vlue + '% ' + sign +')'
        
        text += type + ': ' + str(int(round(sum[0],0)))  + ' руб.' + mean_change + '\n'

    text += '\nИтого на сумму ' + str(int(round(all_sum,0))) + ' руб.\n'

else:
    text = 'За текущий месяц трат не найдено...'

print(text)   

Ваши траты в текущем месяце: 
сбережения: 17264 руб.
еда: 2012 руб.( на 177% more)
другие: 700 руб.( на 0% less)
личные расходы: 490 руб.
транспорт: 20 руб.( на 92% less)

Итого на сумму 20486 руб.



In [4]:
df_all_prev_month_expense

df_all_prev_month_expense_group = DataFrame(data = df_all_prev_month_expense.groupby(['transaction_type'])['summa'].sum() ).sort_values(by='summa' , ascending=False)

df_all_prev_month_expense['year'] = df_all_prev_month_expense['date_fact'].dt.year
df_all_prev_month_expense['month'] = df_all_prev_month_expense['date_fact'].dt.month
df_all_prev_month_expense['yearmonth'] = df_all_prev_month_expense['year'].astype(str) +  df_all_prev_month_expense['month'].astype(str)

df_all_prev_month_expense_group = DataFrame(df_all_prev_month_expense.groupby(['transaction_type' , 'yearmonth'])['summa'].sum() ).reset_index()
df_all_prev_month_expense_group['summa'] = df_all_prev_month_expense_group['summa'].astype(float)
df_mean_expense = DataFrame(df_all_prev_month_expense_group.groupby(['transaction_type'])['summa'].mean()).reset_index()

change = sum / df_mean_expense[(df_mean_expense.transaction_type == type)]['summa'].values[0]

sign = 'less' if change < 1. else 'more'
change_vlue = int( (1. - change if sign == 'less' else change - 1.) * 100 )

In [9]:
mean_sum = df_mean_expense['summa'].values.sum()
mean_sign = 'less' if mean_sum > all_sum else 'more'
change_mean = all_sum - mean_sum if mean_sign == 'more' else mean_sum - all_sum

'more'